In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geoip2.database
import myLogReader as mlr
import re
import os

%matplotlib inline

## Automate - Read and Prep log data into DF

#### 1-Define functions

In [2]:
def getListOfFiles(dirName):
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    
    for file in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, file)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
    return allFiles

def readLog(file):
    log_df = pd.read_csv(file
            #,skiprows=[0,1,2,3]
            , comment='#'
            , sep=' ' 
            , usecols=[0,1, 2, 5, 6, 7, 8, 9, 10,11,12,14]
            , na_values='-'
            , names=['date'
                    ,'time'
                    ,'server-ip'
                    ,'cs-uri-query'
                    ,'server-port'
                    ,'cs-username'
                    ,'client-ip'
                    ,'cs(User-Agent)'
                    ,'cs(Referer)'
                    ,'sc-status'
                    ,'sc-substatus'
                   ,'time-taken(ms)'])
    return log_df

def deriveCityFromIP(iis_log_df):
    #Enhance dataset with Client city and country , derived from IP
    iis_log_df['client-city'] =  iis_log_df['client-ip'].apply(lambda x: reader.city(ip_address=x).city.name if reader.city(ip_address=x).city.name != None else np.nan)
    return iis_log_df
    
def deriveCountryFromIP(iis_log_df):
    iis_log_df['client-country'] =  iis_log_df['client-ip'].apply(lambda x: reader.city(ip_address=x).country.name if reader.city(ip_address=x).country.name != None else np.nan)
    return iis_log_df

def getDevice (UserAgentResponse):
    device ='Other' 
    if 'Mobi' in UserAgentResponse:
        device = 'Mobile'
    else:
        device = 'Desktop'
    return device

def getBrowser (UserAgentResponse):
    browser ='Other' 
    if 'Firefox' in UserAgentResponse and 'Seamonkey' not in UserAgentResponse:
        browser = 'Firefox'
    elif 'Seamonkey' in UserAgentResponse:
        browser = 'Seamonkey'
    elif 'Chrome' in UserAgentResponse and 'Chromium' not in UserAgentResponse:
        browser = 'Chrome'
    elif ('Safari' in UserAgentResponse and 'Chromium' not in UserAgentResponse and 'Chrome' not in UserAgentResponse):
        browser = 'Safari'
    elif 'OPR' in UserAgentResponse and 'Opera'  in UserAgentResponse:
        browser = 'Opera'
    elif '; MSIE' in UserAgentResponse:
        browser = 'IE'
    return browser

def GetWebPageSection(x):
    section = 'Unknown'
    r = re.compile('([A-Z])\w+')
    section = r.search(x)
    if section is not None:
        return section.group()
    return section

def deriveClientDevice(iis_log_df):
    iis_log_df['client-device']  =  iis_log_df['cs(User-Agent)'].apply(lambda x: getDevice(str(x)))
    return iis_log_df

def deriveClientBrowser(iis_log_df):
    iis_log_df['client-browser'] =  iis_log_df['cs(User-Agent)'].apply(lambda x: getBrowser(str(x)))
    return iis_log_df

def deriveClientWebPage(iis_log_df):
    iis_log_df['client-webPage'] = iis_log_df['cs(Referer)'].apply(lambda x: GetWebPageSection(x) if type(x) != float else np.nan)
    return iis_log_df

def deriveClientCity(iis_log_df):
    iis_log_df['client-city'] =  iis_log_df['client-ip'].apply(lambda x: reader.city(ip_address=x).city.name if reader.city(ip_address=x).city.name != None else np.nan)
    return iis_log_df
    
def deriveClientCountry(iis_log_df):    
    iis_log_df['client-country'] =  iis_log_df['client-ip'].apply(lambda x: reader.city(ip_address=x).country.name if reader.city(ip_address=x).country.name != None else np.nan)
    return iis_log_df

In [3]:
dirName = '../data/logs';
reader = geoip2.database.Reader('../data/GeoLite2-City_20181009/GeoLite2-City.mmdb')
#reader.close()
 
# Get the list of all files in directory tree at given path
listOfFiles = getListOfFiles(dirName)

df = readLog(listOfFiles[1])

df = deriveCityFromIP(df)

df = deriveCountryFromIP(df)

df = deriveClientDevice(df)
df = deriveClientBrowser(df)
df = deriveClientWebPage(df)

df = deriveClientCity(df)
df = deriveClientCountry(df)

df.head()

,date,time,server-ip,cs-uri-query,server-port,cs-username,client-ip,cs(User-Agent),cs(Referer),sc-status,sc-substatus,time-taken(ms),client-city,client-country,client-device,client-browser,client-webPage
0,2017-12-08,03:40:50,192.168.2.210,NaN,443,NaN,144.139.133.243,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64;+rv:5...,NaN,302,0,140,Melbourne,Australia,Desktop,Firefox,NaN
1,2017-12-08,03:40:50,192.168.2.210,ReturnUrl=%2f,443,NaN,144.139.133.243,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64;+rv:5...,NaN,302,0,46,Melbourne,Australia,Desktop,Firefox,NaN
2,2017-12-08,03:40:51,192.168.2.210,ReturnUrl=%2f,443,NaN,144.139.133.243,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64;+rv:5...,NaN,200,0,78,Melbourne,Australia,Desktop,Firefox,NaN
3,2017-12-08,03:40:51,192.168.2.210,v=ZLF68Gqwmuuh2ZvHcQpEuU1xkWptxwOpRzXwwMGaiN01,443,NaN,144.139.133.243,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64;+rv:5...,https://canberra-prodapp1.inplace.com.au/Secur...,200,0,171,Melbourne,Australia,Desktop,Firefox,Security
4,2017-12-08,03:40:51,192.168.2.210,v=84kf7GIBdMhYIINVmeCDSHFVxrD5iToJR-sVWXPTNo81,443,NaN,144.139.133.243,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64;+rv:5...,https://canberra-prodapp1.inplace.com.au/Secur...,200,0,46,Melbourne,Australia,Desktop,Firefox,Security


#### 2- Automate load logs and aggregate data

In [5]:
df.head()

Index(['date', 'time', 'server-ip', 'cs-uri-query', 'server-port',
       'cs-username', 'client-ip', 'cs(User-Agent)', 'cs(Referer)',
       'sc-status', 'sc-substatus', 'time-taken(ms)', 'client-city',
       'client-country', 'client-device', 'client-browser', 'client-webPage'],
      dtype='object')

In [486]:
client_browser_df = (df.groupby(by=['date','client-browser'])['client-browser']
                        .count()
                        .reset_index(level=1,name='Count')
                        .pivot(columns='client-browser',values='Count'))

client_city_df = (df.groupby(by=['date','client-city'])['client-city']
            .count()
            .reset_index(level=1,name='Count')
            .pivot(columns='client-city',values='Count'))

client_country_df = (df.groupby(by=['date','client-country'])['client-country']
            .count()
            .reset_index(level=1,name='Count')
            .pivot(columns='client-country',values='Count'))

client_device_df = (df.groupby(by=['date','client-device'])['client-device']
            .count()
            .reset_index(level=1,name='Count')
            .pivot(columns='client-device',values='Count'))

client_webPage_df = (df.groupby(by=['date','client-webPage'])['client-webPage']
            .count()
            .reset_index(level=1,name='Count')
            .pivot(columns='client-webPage',values='Count'))

(df.groupby(by=['date','client-ip'])['client-ip']
            .count()
            .reset_index(level=1,name='Count')
            .pivot(columns='client-ip',values='Count'))

client-ip,144.139.133.243
date,
2017-12-08,211


In [489]:
f.groupby(by=['date','client-ip'])['client-ip']

<bound method Series.unique of count                 211
unique                  1
top       144.139.133.243
freq                  211
Name: client-ip, dtype: object>

In [481]:
df.iloc[10:20,[0,5,6,11,12,13,14,15,16]]

,date,cs-username,client-ip,time-taken(ms),client-city,client-country,client-device,client-browser,client-webPage
10,2017-12-08,NaN,144.139.133.243,46,Melbourne,Australia,Desktop,Firefox,Security
11,2017-12-08,NaN,144.139.133.243,640,Melbourne,Australia,Desktop,Firefox,Security
12,2017-12-08,NaN,144.139.133.243,0,Melbourne,Australia,Desktop,Firefox,Security
13,2017-12-08,NaN,144.139.133.243,875,Melbourne,Australia,Desktop,Firefox,Security
14,2017-12-08,NaN,144.139.133.243,78,Melbourne,Australia,Desktop,Firefox,Security
15,2017-12-08,NaN,144.139.133.243,31,Melbourne,Australia,Desktop,Firefox,Security
16,2017-12-08,NaN,144.139.133.243,78,Melbourne,Australia,Desktop,Firefox,Security
17,2017-12-08,NaN,144.139.133.243,1062,Melbourne,Australia,Desktop,Firefox,Security
18,2017-12-08,NaN,144.139.133.243,2328,Melbourne,Australia,Desktop,Firefox,Security
19,2017-12-08,NaN,144.139.133.243,31,Melbourne,Australia,Desktop,Firefox,Security


In [371]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 17 columns):
date              211 non-null object
time              211 non-null object
server-ip         211 non-null object
cs-uri-query      116 non-null object
server-port       211 non-null int64
cs-username       160 non-null object
client-ip         211 non-null object
cs(User-Agent)    211 non-null object
cs(Referer)       204 non-null object
sc-status         211 non-null int64
sc-substatus      211 non-null int64
time-taken(ms)    211 non-null int64
client-city       211 non-null object
client-country    211 non-null object
client-device     211 non-null object
client-browser    211 non-null object
client-webPage    196 non-null object
dtypes: int64(4), object(13)
memory usage: 28.1+ KB


In [349]:
iis_logs_df.head()

,date,time,server-ip,cs-uri-query,server-port,cs-username,client-ip,cs(User-Agent),cs(Referer),sc-status,sc-substatus,time-taken(ms)
0,2018-10-10,00:00:19,192.168.2.210,NaN,80,NaN,58.167.88.59,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+Appl...,NaN,301,0,93
1,2018-10-10,00:00:19,192.168.2.210,NaN,443,NaN,58.167.88.59,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+Appl...,NaN,302,0,93
2,2018-10-10,00:00:19,192.168.2.210,ReturnUrl=%2f,443,NaN,58.167.88.59,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+Appl...,NaN,302,0,93
3,2018-10-10,00:00:19,192.168.2.210,ReturnUrl=%2f,443,NaN,58.167.88.59,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+Appl...,NaN,200,0,171
4,2018-10-10,00:00:20,192.168.2.210,v=pIhLJRZ-i9xD3brYZ6I5pZD6973WqBDt6kYnAg5j20E1,443,NaN,58.167.88.59,Mozilla/5.0+(Windows+NT+10.0;+Win64;+x64)+Appl...,https://inplace.canberra.edu.au/Security/Accou...,200,0,265


In [363]:
iis_logs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12334 entries, 0 to 12333
Data columns (total 12 columns):
date              12334 non-null object
time              12334 non-null object
server-ip         12334 non-null object
cs-uri-query      5510 non-null object
server-port       12334 non-null int64
cs-username       7275 non-null object
client-ip         12334 non-null object
cs(User-Agent)    12138 non-null object
cs(Referer)       9206 non-null object
sc-status         12334 non-null int64
sc-substatus      12334 non-null int64
time-taken(ms)    12334 non-null int64
dtypes: int64(4), object(8)
memory usage: 1.1+ MB


In [153]:
iis_logs_df.describe()

,server-port,sc-status,sc-substatus,time-taken(ms)
count,12334.000000,12334.000000,12334.0,12334.000000
mean,442.323091,215.232447,0.0,532.415518
std,15.661410,38.194092,0.0,2264.506444
min,80.000000,200.000000,0.0,0.000000
25%,443.000000,200.000000,0.0,62.000000
50%,443.000000,200.000000,0.0,187.000000
75%,443.000000,200.000000,0.0,330.000000
max,443.000000,404.000000,0.0,82662.000000


In [158]:
iis_logs_df["server-port"].describe()

count    12334.000000
mean       442.323091
std         15.661410
min         80.000000
25%        443.000000
50%        443.000000
75%        443.000000
max        443.000000
Name: server-port, dtype: float64

In [167]:
iis_logs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12334 entries, 0 to 12333
Data columns (total 16 columns):
date              12334 non-null object
time              12334 non-null object
server-ip         12334 non-null object
cs-uri-query      5510 non-null object
server-port       12334 non-null int64
cs-username       7275 non-null object
client-ip         12334 non-null object
cs(User-Agent)    12138 non-null object
cs(Referer)       9206 non-null object
sc-status         12334 non-null int64
sc-substatus      12334 non-null int64
time-taken(ms)    12334 non-null int64
client-city       12208 non-null object
client-country    12334 non-null object
client-device     12334 non-null object
client-browser    12334 non-null object
dtypes: int64(4), object(12)
memory usage: 1.5+ MB


## Derive Calendar Week, year after aggregating the data

In [327]:
iis_logs_df['date-weekday']   = iis_logs_df['date'].apply(lambda x: 'Weekday' if np.int8(str(dt.datetime.strptime(x,'%Y-%m-%d').weekday())) < 5 else 'Weekend')
iis_logs_df['date-calendar-week']   = iis_logs_df['date'].apply(lambda x: np.int8(str(dt.datetime.strptime(x,'%Y-%m-%d').isocalendar()[1])))
iis_logs_df['date-year']   = iis_logs_df['date'].apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d').year)

In [328]:
iis_logs_df.loc[:, iis_logs_df.dtypes == object].describe()

,date,time,server-ip,cs-uri-query,cs-username,client-ip,cs(User-Agent),cs(Referer),client-city,client-country,client-device,client-browser,client-webPage,date-weekday
count,12334,12334,12334,5510,7275,12334,12138,9206,12208,12334,12334,12334,7855,12334
unique,1,6220,1,2716,84,222,90,313,66,14,2,4,17,1
top,2018-10-10,02:56:42,192.168.2.210,ReturnUrl=%2f,s437875,137.92.20.64,Mozilla/5.0+(Windows+NT+10.0;+WOW64)+AppleWebK...,https://inplace.canberra.edu.au/Security/Accou...,Canberra,Australia,Desktop,Chrome,Maintenance,Weekday
freq,12334,27,12334,201,1102,993,2289,1242,4119,10003,11653,8129,4005,12334


In [326]:
import datetime as dt
d =dt.datetime.strptime(iis_logs_df.date.loc[1],'%Y-%m-%d')
d =dt.datetime.strptime('2018-12-28','%Y-%m-%d')
d.year
#np.int8(str(d.year))

#iis_logs_df.date.loc[1]

2018

In [228]:
iis_logs_df.isna().sum()

date                 0
time                 0
server-ip            0
cs-uri-query      6824
server-port          0
cs-username       5059
client-ip            0
cs(User-Agent)     196
cs(Referer)       3128
sc-status            0
sc-substatus         0
time-taken(ms)       0
client-city        126
client-country       0
client-device        0
client-browser       0
client-webPage    4479
dtype: int64